/* PART 2: SQLite

Export the country club data from PHPMyAdmin, and connect to a local SQLite instance from Jupyter notebook 
for the following questions.  

In [1]:
import sqlite3 
import pandas as pd
from sqlite3 import Error
from sqlalchemy import create_engine

In [2]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT *
        FROM FACILITIES
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)


In [3]:
#creating engine
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

In [4]:
#printing table names
table_names = engine.table_names()
print(table_names)

['Bookings', 'Facilities', 'Members']


In [5]:

con = sqlite3.connect('sqlite_db_pythonsqlite.db')
cur = con.cursor()
con

/* Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! */

In [6]:
Q10 = pd.read_sql_query("SELECT f.name, SUM(CASE WHEN b.memid = 0\
                             THEN b.slots * f.guestcost ELSE b.slots * f.membercost \
                             END) AS total_revenue\
                             FROM Bookings AS b \
                             INNER JOIN Facilities AS f \
                             ON b.facid = f.facid \
                             GROUP BY f.name \
                             HAVING total_revenue < 1000",engine)
Q10.head()

,name,total_revenue
0,Pool Table,270
1,Snooker Table,240
2,Table Tennis,180


/* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */

In [10]:
Q11 = pd.read_sql_query("SELECT m.surname||' '|| m.firstname As Member,\
                             n.surname||' '|| n.firstname AS Recommedendedby \
                             FROM Members AS m, Members AS n \
                             WHERE m.memid= n.recommendedby AND m.recommendedby !=0 \
                             OR n.memid= m.recommendedby AND m.recommendedby !=0 \
                             ORDER BY m.surname;", engine) 
Q11.head()

,Member,Recommedendedby
0,Bader Florence,Stibbons Ponder
1,Bader Florence,Sarwin Ramnaresh
2,Baker Anne,Stibbons Ponder
3,Baker Timothy,Farrell Jemima
4,Baker Timothy,Coplin Joan


/* Q12: Find the facilities with their usage by member, but not guests */

In [8]:
Q12 = pd.read_sql_query("SELECT f.name AS facility_name, COUNT(b.starttime) AS usage \
FROM Bookings AS b \
INNER JOIN Facilities AS f \
ON b.facid = f.facid \
WHERE b.memid !=0 \
GROUP BY f.name", engine)

Q12

,facility_name,usage
0,Badminton Court,344
1,Massage Room 1,421
2,Massage Room 2,27
3,Pool Table,783
4,Snooker Table,421
5,Squash Court,195
6,Table Tennis,385
7,Tennis Court 1,308
8,Tennis Court 2,276


/* Q13: Find the facilities usage by month, but not guests */

In [9]:
Q13 = pd.read_sql_query("SELECT f.name AS facility_name,\
(strftime('%Y-%m', starttime)) as usage_by_month, \
COUNT(b.starttime) AS total_usage \
FROM Bookings AS b \
INNER JOIN Facilities AS f \
ON b.facid = f.facid \
WHERE b.memid !=0 \
GROUP BY f.name, usage_by_month", engine)

Q13

,facility_name,usage_by_month,total_usage
0,Badminton Court,2012-07,51
1,Badminton Court,2012-08,132
2,Badminton Court,2012-09,161
3,Massage Room 1,2012-07,77
4,Massage Room 1,2012-08,153
5,Massage Room 1,2012-09,191
6,Massage Room 2,2012-07,4
7,Massage Room 2,2012-08,9
8,Massage Room 2,2012-09,14
9,Pool Table,2012-07,103
